In [88]:
import csv
with open('enjoySport.csv') as csvFile:
  examples = [tuple(line)[1:] for line in csv.reader(csvFile)]

examples = examples[1:]
examples

[('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Y'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Y'),
 ('Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change', 'N'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change', 'Y')]

In [89]:
def g_0(n):
  return ("?",)*n

def s_0(n):
  return ("0",)*n

In [90]:
def get_domains(examples):
  d = [set() for x in examples[0]]
  for x in examples:
    for i, xi in enumerate(x):
      d[i].add(xi)
  
  return [sorted(elem) for elem in d]

# get_domains(examples)

In [91]:
def more_general(h1, h2):
  all_flags = []
  for h,x in zip(h1, h2):
    flag = h == "?" or (h!="0" and (h==x  or x=="0"))
    all_flags.append(flag)
  
  return all(all_flags)

In [92]:
def min_generalisation(h, x):
  h_new = list(h)
  for i in range(len(h)):
    if not more_general(h[i:i+1], x[i:i+1]):
      h_new[i] = "?" if h[i] != "0" else x[i]
  return [tuple(h_new)]

In [93]:
def min_specialisation(h, x, domains):
  result = []
  for i in range(len(h)):
    if h[i] == "?":
      for val in domains[i]:
        if val != x[i]:
          h_new = h[:i] + (val,) + h[i+1:]
          result.append(h_new)
    elif h[i] != "0":
      h_new = h[:i] + ("0",) + h[i+1:]
      result.append(h_new)
    
  return result

In [94]:
def generalise_S(x, S, G):
  S_prev = list(S)
  for s in S_prev:
    if s not in S:
      continue
    if not more_general(s, x):
      S.remove(s)
      Splus = min_generalisation(s, x)

      S.update([h for h in Splus if any([more_general(g,h) for g in G])])
      S.difference_update([h for h in S if any([more_general(h, h1) for h1 in S if h!=h1])])
  
  return S


In [95]:
def specialise_G(x, S, G, domains):
  G_prev = list(G)
  for g in G_prev:
    if g not in G:
      continue
    if more_general(g, x):
      G.remove(g)
      G_minus = min_specialisation(g, x, domains)

      G.update([h for h in G_minus if any([more_general(h,s) for s in S])])
      G.difference_update([h for h in G if any([more_general(h1, h) for h1 in G if h!=h1])])

  return G

In [96]:
domains = get_domains(examples)[:-1]
G = set([g_0(len(domains))])
S = set([s_0(len(domains))])
i = 0
print(f'G-{i} -> {G}')
print(f'S-{i} -> {S}')

for xcx in examples:
  i+=1
  x, label = xcx[:-1], xcx[-1]
  if label == "Y":
    G = {g for g in G if more_general(g, x)}
    S = generalise_S(x, S, G)
  elif label == "N":
    S = {s for s in S if not more_general(s, x)}
    G = specialise_G(x, S, G, domains)
  
  print(f'G-{i} -> {G}')
  print(f'S-{i} -> {S}')


G-0 -> {('?', '?', '?', '?', '?', '?')}
S-0 -> {('0', '0', '0', '0', '0', '0')}
G-1 -> {('?', '?', '?', '?', '?', '?')}
S-1 -> {('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same')}
G-2 -> {('?', '?', '?', '?', '?', '?')}
S-2 -> {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}
G-3 -> {('?', '?', '?', '?', '?', 'Same'), ('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}
S-3 -> {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}
G-4 -> {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}
S-4 -> {('Sunny', 'Warm', '?', 'Strong', '?', '?')}
